In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import typing
import data_wrangling as dw
import canvas

# this enable matplotlib to show chinese letters
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS']

In [2]:
df = dw.ds_read('點餐明細20221017.csv')
df.head()

Using file name: "點餐明細20221017.csv" to transform the data


,交易序號,發票編號,類別名稱,商品名稱,數量,點單金額,客數,商品單位
0,20220901 02020001,NaN,招待,員工喝 Redbull,1,0,0,選擇追蹤單位
1,20220901 02020001,NaN,杯子,破杯費(薄威士忌杯),1,0,0,選擇追蹤單位
2,20220901 02020001,NaN,招待,招待 jagermeister Shot,3,0,0,NaN
3,20220901 02020001,NaN,杯子,破杯費(大馬丁尼杯),1,0,0,選擇追蹤單位
4,20220901 02020001,NaN,招待,員工喝High Ball,4,0,0,選擇追蹤單位


In [3]:
df_clean = dw.drop_na_col(df, 0.7)
df_clean = df_clean.drop(columns=['客數', '商品單位'])
df_clean.columns = ['sales_date', 'type', 'product', 'quan', 'price']
print(df_clean.shape)
df_clean.head()


data NA Summary: --------------------------------------------
發票編號    0.887885
dtype: float64
--------------------------------------------
droping columns: ['發票編號']
(3211, 5)


,sales_date,type,product,quan,price
0,20220901 02020001,招待,員工喝 Redbull,1,0
1,20220901 02020001,杯子,破杯費(薄威士忌杯),1,0
2,20220901 02020001,招待,招待 jagermeister Shot,3,0
3,20220901 02020001,杯子,破杯費(大馬丁尼杯),1,0
4,20220901 02020001,招待,員工喝High Ball,4,0


In [13]:
df_clean.loc[:, 'sales_date'] = df_clean.sales_date.apply(lambda x: x[:-4])
df_clean.head()

,sales_date,type,product,quan,price
0,20220901 0202,招待,員工喝 Redbull,1,0
1,20220901 0202,杯子,破杯費(薄威士忌杯),1,0
2,20220901 0202,招待,招待 jagermeister Shot,3,0
3,20220901 0202,杯子,破杯費(大馬丁尼杯),1,0
4,20220901 0202,招待,員工喝High Ball,4,0


In [22]:
a, b, c, d = dw.deal_date(df_clean.sales_date, '%Y%m%d %H%M')

0        2
1        2
2        2
3        2
4        2
        ..
3206    17
3207    17
3208    17
3209    17
3210    17
Name: sales_date, Length: 3211, dtype: int64

In [27]:
df_all_date = pd.DataFrame({
    'date': b,
    'hour': c,
    'weekday_name': d
})

df_all_date.head()

,date,hour,weekday_name
0,2022-09-01,2,Thu
1,2022-09-01,2,Thu
2,2022-09-01,2,Thu
3,2022-09-01,2,Thu
4,2022-09-01,2,Thu


In [29]:
df_new = pd.concat([df_clean, df_all_date], axis=1)
df_new = df_new.iloc[:, 1:]

In [30]:
df_new

,type,product,quan,price,date,hour,weekday_name
0,招待,員工喝 Redbull,1,0,2022-09-01,2,Thu
1,杯子,破杯費(薄威士忌杯),1,0,2022-09-01,2,Thu
2,招待,招待 jagermeister Shot,3,0,2022-09-01,2,Thu
3,杯子,破杯費(大馬丁尼杯),1,0,2022-09-01,2,Thu
4,招待,員工喝High Ball,4,0,2022-09-01,2,Thu
...,...,...,...,...,...,...,...
3206,Beer,PERONI draft beer,1,250,2022-09-30,17,Fri
3207,主食,干貝雞汁拌麵,1,260,2022-09-30,17,Fri
3208,Signature Cocktail,琥珀,1,380,2022-09-30,17,Fri
3209,Beer,PERONI draft beer,1,250,2022-09-30,17,Fri


In [32]:
df_top_3_type = dw.get_top_n_filter(df_new, 'type', 'price')
df_top_3_type.head()

Top 3 sum of 'price' from 'type' column:

type
Signature Cocktail    392450
Special及活動             69810
炸物                     65340
Name: price, dtype: int64
--------------------------------------------
Filtering data by the column: type values...['Signature Cocktail' 'Special及活動' '炸物']


,type,product,quan,price,date,hour,weekday_name
12,Signature Cocktail,西方世界,1,400,2022-09-01,21,Thu
13,Signature Cocktail,尋根,1,400,2022-09-01,21,Thu
14,Signature Cocktail,浮生若夢,1,400,2022-09-01,20,Thu
18,Signature Cocktail,雲嫣,1,380,2022-09-01,22,Thu
19,炸物,可樂果炸雞,1,320,2022-09-01,22,Thu


In [116]:
df_top_3_agg = df_top_3_type.groupby(['date', 'weekday_name', 'type', 'product'])['price'].sum().reset_index()
df_top_3_agg['is_weekend'] = df_top_3_agg.weekday_name.isin(['Fri', 'Sat'])
df_top_3_agg.head()

,date,weekday_name,type,product,price,is_weekend
0,2022-09-01,Thu,Signature Cocktail,尋根,800,False
1,2022-09-01,Thu,Signature Cocktail,浮生若夢,400,False
2,2022-09-01,Thu,Signature Cocktail,琥珀,380,False
3,2022-09-01,Thu,Signature Cocktail,西方世界,400,False
4,2022-09-01,Thu,Signature Cocktail,雲初之上,400,False


In [134]:
df_top_3_agg.groupby(['date', 'weekday_name', 'is_weekend', 'type'])['price'].sum().unstack()['炸物']

date        weekday_name  is_weekend
2022-09-01  Thu           False          540.0
2022-09-02  Fri           True          4960.0
2022-09-03  Sat           True          2230.0
2022-09-04  Sun           False         1060.0
2022-09-06  Tue           False         1000.0
2022-09-07  Wed           False            NaN
2022-09-08  Thu           False         2850.0
2022-09-09  Fri           True          1520.0
2022-09-10  Sat           True          2520.0
2022-09-11  Sun           False          220.0
2022-09-12  Mon           False         1940.0
2022-09-13  Tue           False            NaN
2022-09-14  Wed           False         2740.0
2022-09-15  Thu           False         1180.0
2022-09-16  Fri           True          7120.0
2022-09-17  Sat           True          4070.0
2022-09-18  Sun           False          220.0
2022-09-19  Mon           False         1210.0
2022-09-20  Tue           False          740.0
2022-09-21  Wed           False         2490.0
2022-09-22  Thu        

In [181]:
# What is the top 3 sales of each these three types?
df_top_3_agg.groupby(['type', 'product'])['price'].sum()

type                product                  
Signature Cocktail  The MACALLAN Signature        4950
                    The Macallan Highball         1400
                    The Macallan Silend Third     1200
                    哈庫那瑪塔塔                       39480
                    尋根                           50800
                    山海關                          34860
                    東方美人                         37380
                    浪子之琴                         15120
                    浮生若夢                         10800
                    炙陽                           11400
                    琥珀                           69160
                    白霜                           19760
                    綠洲                            6800
                    西方世界                         13600
                    雲初之上                         23600
                    雲嫣                           20140
                    龍柏                           32000
Special及活動         

In [240]:
df_top_three_final = df_top_3_agg.pivot_table(index=['date', 'weekday_name', 'is_weekend', 'type'],
                         values='price',
                         aggfunc={'price': np.sum}).reset_index()

display(df_top_three_final)

# sns.lineplot(data=df_top_three_final, x='date', y='price', errorbar=None, estimator=None, hue='type')
# plt.xticks(rotation=45)

,date,weekday_name,is_weekend,type,price
0,2022-09-01,Thu,False,Signature Cocktail,3160
1,2022-09-01,Thu,False,炸物,540
2,2022-09-02,Fri,True,Signature Cocktail,30300
3,2022-09-02,Fri,True,Special及活動,7770
4,2022-09-02,Fri,True,炸物,4960
...,...,...,...,...,...
80,2022-09-30,Fri,True,Special及活動,2530
81,2022-09-30,Fri,True,炸物,7930
82,2022-10-01,Sat,True,Signature Cocktail,1620
83,2022-10-01,Sat,True,Special及活動,800
